#### Project 3: The worm algorithm for the 6-vertex model (Monte Carlo)

Implement the worm algorithm for the 6-vertex (ice-type) model (https://en.wikipedia.org/wiki/Ice-type_model) at $T = \infty$ (where all allowed configurations are equally likely).

Calculate the exponent $a$ of the correlation function $\langle s_0 s_r \rangle \propto \frac{1}{r^a}$.

In [196]:
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline

In [197]:
class six_vertex_model(object):
    """Ice-type (6-vertex) model class with periodic boundary conditions.
    
    Attributes
    ----------
    Lx : int
        Lattice X size.
    Ly : int
        Lattice Y size.
    N : int
        Number of vertices.
    lattice : list
        Horizontal (hor) & vertical (ver) sublattices.
        Shape: ((Ly, Lx), (Lx, Ly)).
    
    Methods
    ----------
    - plot()
    - energy(energies)
    - get_vertex_in_out(x, y)
    - get_vertex_charge(x, y)
    - get_next_vertex(x, y, flip)
    - flip_spin(x, y, flip)
    - flip_next(x, y, flip)
    - check_valid_config()
    - propagate_2_defects()
    - thermalize(N)
    - measure_correlations()
    """
    
    def __init__(self, Lx, Ly, hor=[], ver=[]): 
        '''hor, ver: initial lattice configuration.
        '''
        self.Lx = Lx
        self.Ly = Ly
        self.N  = Lx * Ly;
        
        # periodic BC: hor[_][0] = hor[_][Lx+1] && ver[_][0] = ver[_][Ly+1]
        if hor == [] or ver == []:
            hor = np.ones((Ly, Lx), dtype=np.int8) 
            ver = np.ones((Lx, Ly), dtype=np.int8)
            # initial state: alternate directions from line to line
            hor[1::2] *= -1
            ver[1::2] *= -1
            self.lattice = np.array([hor, ver]) # shape: ((Ly, Lx), (Lx, Ly))
        else:
            assert hor.shape == (Ly, Lx) and ver.shape == (Lx, Ly)
            self.lattice = np.array([hor, ver]) # shape: ((Ly, Lx), (Lx, Ly))
            self.check_valid_config()
        
        # self.n = np.array([?]*6) # (dummy) how many in each of the 6 config types
    
    def plot(self):
        '''Plot hor & ver sublattices (´ver´ transposed for graphical purposes).
        '''
        print('Horizontal spins:')
        print(self.lattice[0], '\n')
        print('Vertical spins:')
        print(self.lattice[1].transpose(), '\n')
    
    def energy(self, energies=[1.]*6):
        '''(dummy) energies : list with energies of the 6 configurations.
        '''
        return self.N # T->Inf: all equally likely
        # return np.sum(self.n * np.array(energies))

    def get_vertex_in_out(self, x, y):
        '''Return spin values: np.array([spin_left, spin_right, spin_up, spin_down])
        relative to vertex (x, y).
        In  :  1
        Out : -1
        '''
        spin_left  = self.lattice[0, x, y]
        spin_right = self.lattice[0, x, (y+1) % self.Lx] * -1
        spin_up    = self.lattice[1, y, x] * -1
        spin_down  = self.lattice[1, y, (x+1) % self.Ly]
        return np.array([spin_left, spin_right, spin_up, spin_down])
    
    def get_vertex_charge(self, x, y):
        '''Return magnetic charge of vertex (x, y).
        '''
        return np.sum(self.get_vertex_in_out(x, y)) / 2

    def get_next_vertex(self, x, y, flip):
        '''Return coord of neighbour vertex in the flip direction & flip
        relative to that vertex.
        x, y : 1st vertex coord
        flip : 0 -> left, 1 -> right, 2 -> up, 3 -> down
        '''
        if flip == 0:   # left
            return x, (y-1) % self.Lx, 1
        elif flip == 1: # right
            return x, (y+1) % self.Lx, 0
        elif flip == 2: # up
            return (x-1) % self.Ly, y, 3
        else:           # down
            return (x+1) % self.Ly, y, 2
    
    def flip_spin(self, x, y, flip):
        '''Create 2 defects by flipping 1 spin.
        x, y : vertex coord
        flip : 0 -> left, 1 -> right, 2 -> up, 3 -> down
        '''
        if flip == 0:   # left
            self.lattice[0, x, y] *= -1
        elif flip == 1: # right
            self.lattice[0, x, (y+1) % self.Lx] *= -1
        elif flip == 2: # up
            self.lattice[1, y, x] *= -1
        else:           # down
            self.lattice[1, y, (x+1) % self.Ly] *= -1
    
    def flip_next(self, x, y, flipped):
        '''Propagate defect by flipping randomly one of the vertex's other spins,
        subject to the ice rule (2-in, 2-out).
        Return next vertex coord & new flip relative to that vertex.
        x, y    : vertex coord
        flipped : 0 -> left, 1 -> right, 2 -> up, 3 -> down
        '''
        in_or_out = self.get_vertex_in_out(x, y)
        flipped_dir = in_or_out[flipped] # in: 1, out: -1
        
        in_or_out[flipped] = 0 # remove old direction from possibilities
        # possible_mask = (in_or_out == flipped_dir) # boolean mask
        possible = np.arange(4)[in_or_out == flipped_dir] # 2 new flip directions
        # print('2 possible flip directions:', possible)
        
        flip_new = -1
        prob = np.random.random() # 50/50 chance
        if prob < 0.5:
            flip_new = possible[0]
        else:
            flip_new = possible[1]
        # print('Chosen direction:', flip_new)
        
        self.flip_spin(x, y, flip_new)
        return self.get_next_vertex(x, y, flip_new)
    
    def check_valid_config(self):
        '''Check if system respects the ice rule (2-in, 2-out).
        Return number of defects (charged vertices).
        '''
        charged = 0
        for x in range(self.Ly):
            for y in range(self.Lx):
                if self.get_vertex_charge(x, y) != 0:
                    charged += 1
        
        if charged == 0:
            print('6-vertex configuration is valid. No finite-charge vertices.')
        else:
            print('6-vertex configuration is not valid!', charged, 'finite-charge vertices.')
    
    def propagate_2_defects(self):
        '''Flip 1 spin, create 2 defective vertices & propagate one randomly until
        they meet & annihilate each other.
        '''
        x = np.random.randint(self.Lx) # vertex x coord
        y = np.random.randint(self.Ly) # vertex y coord
        flip = np.random.randint(4) # 0: left, 1: right, 2: up, 3: down
        # print('Vertex:    ', x, y, '\tSpin-flip:', flip)
        
        self.flip_spin(x, y, flip)
        xf, yf, _ = self.get_next_vertex(x, y, flip) # end vertex coord (x_f, y_f)
        
        propagate = True
        while propagate:
            x, y, flip = self.flip_next(x, y, flip)
            # print('\nNew vertex:', x, y, '\tNew flip:', flip)
            if (x, y) == (xf, yf): # defects met & annihilated
                propagate = False
    
    def thermalize(self, N=1000):
        '''Thermalize system by running ´propagate_2_defects´ N times.
        '''
        for _ in range(N):
            self.propagate_2_defects()
    
    def measure_correlations():
        '''Measure spin correlations with distance on square lattice.
        Errors decreased using translational & rotational invariance.
        
        Return: corr_along, corr_perp, corr_diag
            corr_along : correlations along spin direction.            Size: L - 1
            corr_perp  : correlations perpendicular to spin direction. Size: L - 1
            corr_diag  : diagonal correlations.                    Size: 2 * L - 1
        
        Types:
            1) corrhor_hor : horizontal correlations of horizontal spins. Size: L - 1
            2) corrver_ver :  vertical  correlations of  vertical  spins. Size: L - 1
            3) corrver_hor :  vertical  correlations of horizontal spins. Size: L - 1
            4) corrhor_ver : horizontal correlations of  vertical  spins. Size: L - 1
            5) corr_diag    :  diagonal  correlations.                Size: 2 * L - 1
            
        Due to rotational invariance:
        - (1) & (2) should be the same.
        - (3) & (4) should be the same.
        '''
        assert self.Lx == self.Ly
        
        def calculate_correlation(lat0, lat1):
            return lat * lat1

        def calculate_correlation(lat, origins, others, axis):
            '''Correlation averaged over all ´origins´ points.
            '''
            return np.sum(lat[origins] * lat[others], axis) / lat.shape[axis]
        
        corr_diag   = np.zeros(2 * np.min([self.Lx, self.Ly]) - 1)
         
        corrhor_hor = calculate_correlation(self.lattice[0], (slice(None), 0, np.newaxis),
                                                             (slice(None), slice(1,None)), axis=0)
        corrver_hor = calculate_correlation(self.lattice[0], 0, slice(1,None), axis=1)
        
        # corrhor_hor = calculate_correlation(self.lattice[0], (:, 0, np.newaxis), (:, 1:),
                                            # axis=0, N=self.lattice[0].shape[0])
        # corrhor_hor = calculate_correlation(self.lattice[0], (0, :), (1:, :),
                                            # axis=0, N=self.lattice[0].shape[0] - 1)
        
        return corrhor_hor, corrver_hor, ver_vercorr, ver_horcorr, corr_diag
    

In [198]:
# Testing:
L = 3
icetype = six_vertex_model(L, L)
# print('Energy:', icetype.energy([1, 1, 1, 1, 1, 1]))
# print('Vertex (0, 0):', icetype.get_vertex_in_out(0, 0))
# print('Other defect:', icetype.get_other_defect(2, 2, 1), '\n')
# icetype.plot()

# x0, y0, flip0 = 2, 2, 1
# icetype.flip_spin(x0, y0, flip0)
# icetype.plot()
# icetype.flip_next(x0, y0, flip0)
# icetype.plot()
# icetype.check_valid_config()

# icetype.propagate_2_defects()
# icetype.thermalize(10000)
# icetype.plot()

icetype.measure_correlations()

In [195]:
a = np.array([[ 1,  12,  13,  14,  15],
              [ 2,  2,  2,  2,  2],
              [ 3,  3, -3,  -3,  -3]])
b = a[0, :,]
c = a[1:, :]
print(a[0, slice(1,None)], '\n')
print(b, '\n\n', c, '\n')
print(b * c, '\n')
print(np.sum(b * c, axis=1) / (a.shape[1]), a.shape)

[12 13 14 15] 

[ 1 12 13 14 15] 

 [[ 2  2  2  2  2]
 [ 3  3 -3 -3 -3]] 

[[  2  24  26  28  30]
 [  3  36 -39 -42 -45]] 

[ 22.  -17.4] (3, 5)


In [138]:
a = np.array([[ 1,  2, -3,  4, 5],
              [ 1,  2,  3,  4, 5],
              [ 1,  2, -3, -4, 5]])
b = a[:, 0, np.newaxis]
c = a[:, 1:]
print(b, '\n\n', c, '\n')
print(b * c, '\n')
print(np.sum(b * c, axis=0) / a.shape[0], a.shape)

[[1]
 [1]
 [1]] 

 [[ 2 -3  4  5]
 [ 2  3  4  5]
 [ 2 -3 -4  5]] 

[[ 2 -3  4  5]
 [ 2  3  4  5]
 [ 2 -3 -4  5]] 

[ 2.         -1.          1.33333333  5.        ] (3, 5)
